In [10]:
import pandas as pd
import numpy as np
import pandas_profiling
from sklearn.model_selection import train_test_split, KFold

In [11]:
from surprise import SVD, SVDpp, SlopeOne, NormalPredictor, NMF, KNNBaseline, KNNBasic, KNNWithMeans, \
                    KNNWithZScore, BaselineOnly, CoClustering, accuracy, Reader, Dataset
from surprise.model_selection import train_test_split, cross_validate, KFold, GridSearchCV

In [3]:
users = pd.read_csv('users.csv')
movies = pd.read_csv('movies.tsv', sep='\t')
ratings = pd.read_csv('ratings.csv')
alldata = pd.read_csv('allData.tsv', sep='\t')
predict = pd.read_csv('predict.csv')

In [8]:
users.head()

,userID,age,gender
0,4,25,M
1,5,56,M
2,6,50,M
3,15,45,M
4,17,25,F
...,...,...,...
2348,6025,25,F
2349,6027,35,F
2350,6029,35,M
2351,6030,25,M


In [9]:
movies.head()

,movieID,name,year,genre1,genre2,genre3
0,1,Toy Story,1995,Animation,Children's,Comedy
1,2,Jumanji,1995,Adventure,Children's,Fantasy
2,3,Grumpier Old Men,1995,Comedy,Romance,NaN
3,4,Waiting to Exhale,1995,Comedy,Drama,NaN
4,5,Father of the Bride Part II,1995,Comedy,NaN,NaN
...,...,...,...,...,...,...
1460,3948,Meet the Parents,2000,Comedy,NaN,NaN
1461,3949,Requiem for a Dream,2000,Drama,NaN,NaN
1462,3950,Tigerland,2000,Drama,NaN,NaN
1463,3951,Two Family House,2000,Drama,NaN,NaN


In [10]:
ratings.head()

,userID,movieID,rating
0,747,1193,5
1,747,661,3
2,747,2804,5
3,747,919,4
4,747,2791,4
...,...,...,...
31615,4362,1784,4
31616,4362,539,2
31617,4362,2011,2
31618,4362,1097,4


In [8]:
alldata.head()

,userID,age,gender,movieID,name,year,genre1,genre2,genre3,rating
0,747,1,F,1193,One Flew Over the Cuckoo's Nest,1975,Drama,NaN,NaN,5
1,747,1,F,661,James and the Giant Peach,1996,Animation,Children's,Musical,3
2,747,1,F,2804,"Christmas Story, A",1983,Comedy,Drama,NaN,5
3,747,1,F,919,"Wizard of Oz, The",1939,Adventure,Children's,Drama,4
4,747,1,F,2791,Airplane!,1980,Comedy,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...
31615,4362,25,F,1784,As Good As It Gets,1997,Comedy,Drama,NaN,4
31616,4362,25,F,539,Sleepless in Seattle,1993,Comedy,Romance,NaN,2
31617,4362,25,F,2011,Back to the Future Part II,1989,Comedy,Sci-Fi,NaN,2
31618,4362,25,F,1097,E.T. the Extra-Terrestrial,1982,Drama,Fantasy,Sci-Fi,4


In [21]:
predict.head()

,userID,movieID,rating
0,3073,2997,0
1,2083,788,0
2,5949,3386,0
3,4227,2683,0
4,3876,3763,0
...,...,...,...
195,1674,2690,0
196,4076,1219,0
197,5570,2245,0
198,4362,1494,0


In [ ]:
alldata.describe()

In [ ]:
alldata.info()

In [ ]:
netflix_data = alldata.profile_report(sort='None', html={'style':{'full_width': True}}, progress_bar=True)
netflix_data.to_file('Netflix.html')
netflix_data

In [44]:
len(pd.unique(ratings[['userID','movieID']].values.ravel()))

3244

In [11]:
ratings_pivot = ratings.pivot(index='userID', columns='movieID', values='rating')

In [12]:
ratings_pivot.fillna(0, inplace=True)

In [13]:
ratings_pivot

movieID,1,2,3,4,5,6,7,8,9,12,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userID,,,,,,,,,,,,,,,,,,,,,
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
def Eval(test, actual):
    merged = test.merge(actual, how='inner', on ='movieID')
    distance = abs(merged['rating_x'] - merged['rating_y'])
    average_dist = sum(distance)/len(distance)
    number_correct = len(merged[merged.rating_x == merged.rating_y])
    percentage_correct = number_correct/len(merged)
    return (average_dist, number_correct, percentage_correct)

In [5]:
simplified_ratings = ratings[['movieID', 'rating']]

In [8]:
#Average
average = pd.DataFrame(ratings.groupby(['movieID']).mean()['rating'])
average_dist, number_correct, percentage_correct = Eval(average.round(), simplified_ratings)
print('Fractional Rating:', average_dist)
print('Integer Rating:', number_correct)
print('Percentage Rating:', percentage_correct)

Fractional Rating: 0.7144845034788109
Integer Rating: 12616
Percentage Rating: 0.39898798228969007


In [7]:
#Average
average = pd.DataFrame(ratings.groupby(['movieID']).mean()['rating'])
average_dist, number_correct, percentage_correct = Eval(average, simplified_ratings)
print('Fractional Rating:', average_dist)
print('Integer Rating:', number_correct)
print('Percentage Rating:', percentage_correct)

Fractional Rating: 0.7461296507056941
Interger Rating: 954
Percentage Rating: 0.0301707779886148


In [110]:
average_df = predict.merge(average, how='left', on='movieID')
average_df.drop(['rating_x','KNN_preds'], axis=1, inplace=True)
average_df.rename(columns={'rating_y':'rating'}, inplace=True)
average_df.to_csv('average_df.csv')
average_df.round().to_csv('average_df_v2.csv')

In [ ]:
# Rating = Weight * Individual Rating + (1- Weight) * Global Rating
weight = 260*(ratings.groupby(['movieID']).count()['rating']/sum(ratings.groupby(['movieID']).count()['rating']))
ind_rating = ratings.groupby(['movieID']).mean()['rating']
global_rating = ratings.mean()['rating']
weighted_avg_rating = pd.DataFrame(weight*ind_rating + (1-weight) * global_rating)

In [ ]:
average_dist, number_correct, percentage_correct = Eval(weighted_avg_rating.round(), simplified_ratings)
print('Fractional Rating:', average_dist)
print('Integer Rating:', number_correct)
print('Percentage Rating:', percentage_correct)

In [ ]:
weighted_average_df = predict.merge(weighted_avg_rating, how='left', on='movieID')
weighted_average_df.drop(['rating_x'], axis=1, inplace=True)
weighted_average_df.rename(columns={'rating_y':'rating'}, inplace=True)
weighted_average_df.to_csv('weighted_average_df2.csv')
weighted_average_df.round().to_csv('weighted_average_df2_v2.csv')

In [ ]:
#Preprocessing Data- Trainset/Testset
trainset, testset = train_test_split(alldata, test_size= 0.2)

In [13]:
reader = Reader(rating_scale=(1,5))

In [14]:
data = Dataset.load_from_df(ratings[['userID', 'movieID', 'rating']], reader)

In [16]:
#SVD - Singular Value Decomposition
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=False)

{'test_rmse': array([0.93560828, 0.92571989, 0.93265994, 0.93816085, 0.93077744]),
 'test_mae': array([0.73895407, 0.72953197, 0.74456171, 0.74370937, 0.73879247]),
 'fit_time': (1.5833005905151367,
  1.5421109199523926,
  1.5810916423797607,
  1.6580519676208496,
  1.5670993328094482),
 'test_time': (0.11610221862792969,
  0.044974327087402344,
  0.04698634147644043,
  0.043970346450805664,
  0.04397153854370117)}

In [19]:
benchmark = []

for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE', 'MAE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_mae')

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


,test_rmse,test_mae,fit_time,test_time
Algorithm,,,,
SVDpp,0.927226,0.733281,14.804723,0.645037
SVD,0.938008,0.743945,1.547180,0.092781
KNNBaseline,0.943140,0.743997,0.305543,1.737900
BaselineOnly,0.933934,0.744278,0.062437,0.105375
SlopeOne,0.982180,0.767964,0.111334,0.505035
KNNWithMeans,0.995473,0.780605,0.336155,2.012041
KNNWithZScore,1.002195,0.783312,0.323054,1.537966
KNNBasic,0.994523,0.786242,0.234134,1.317015
CoClustering,1.015683,0.788388,0.943982,0.058055


In [ ]:
#Memory-based Collaborative Filtering (User) Evaluation
param_grid = {'k': [10, 20, 30, 40, 50, 60, 70],
              'sim_options': {'name': ['msd', 'cosine'],
                              'min_support': [1, 3, 5],
                              'user_based': [True]}
              }

gs = GridSearchCV(KNNBaseline, param_grid, measures=['rmse','mae'], cv=5, refit=True)
gs.fit(data)

print(gs.best_score['rmse'], gs.best_score['mae'])
print(gs.best_params['rmse'], gs.best_params['mae'])

In [ ]:
{'n_factors': 140, 'n_epochs': 90, 'lr_all': 0.003, 'reg_all': 0.1}
KNN_preds = []
for i in range(len(predict)):
    _, _, _, rating_pred, _ = gs_svd.predict(predict.userID[i], predict.movieID[i])
    KNN_preds.append(rating_pred)

In [20]:
KNN_preds = []
for i in range(len(predict)):
    _, _, _, rating_pred, _ = gs.predict(predict.userID[i], predict.movieID[i])
    KNN_preds.append(rating_pred)

predict['KNN_preds'] = KNN_preds

KNN_df = predict.drop(['rating'], axis=1)
KNN_df.rename(columns={'KNN_preds':'rating'}, inplace=True)
KNN_df.to_csv('KNN_rating_v1.csv')
KNN_df.round().to_csv('KNN_rating_v2.csv')

In [18]:
#Memory-based Collaborative Filtering (User) Evaluation
param_grid = {'k': [10, 20, 30, 40, 50, 60, 70],
              'sim_options': {'name': ['msd', 'cosine'],
                              'min_support': [1, 3, 5],
                              'user_based': [True]},
              'bsl_options': {'reg_i': [5, 10, 15, 20],
                             'reg_u': [10, 15, 20, 25],
                             'n_epochs': [10, 15]}
              }

gs = GridSearchCV(KNNBaseline, param_grid, measures=['rmse','mae'], cv=5, refit=True)
gs.fit(data)

print(gs.best_score['rmse'], gs.best_score['mae'])
print(gs.best_params['rmse'], gs.best_params['mae'])

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matr

In [19]:
knn_algo = gs.best_estimator['mae']
cross_validate(knn_algo, data, measures=['rmse', 'mae'], cv=5, verbose =True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9356  0.9264  0.9359  0.9355  0.9476  0.9362  0.0067  
MAE (testset)     0.7355  0.7304  0.7398  0.7367  0.7489  0.7383  0.0061  
Fit time          0.32    0.31    0.31    0.32    0.29    0.31    0.01    
Test time         1.24    1.20    1.21    1.15    1.14    1.19    0.04    


{'test_rmse': array([0.93563372, 0.92641755, 0.93593576, 0.93545425, 0.94762525]),
 'test_mae': array([0.73550004, 0.73038824, 0.7397844 , 0.73665032, 0.74893529]),
 'fit_time': (0.32379603385925293,
  0.31182122230529785,
  0.3138246536254883,
  0.3247973918914795,
  0.2948338985443115),
 'test_time': (1.2356352806091309,
  1.2002925872802734,
  1.2132978439331055,
  1.1463525295257568,
  1.1433401107788086)}

In [ ]:
knn_algo = KNNBasic()
knn_algo.fit(data)
knn_test_pred = knn.test()

In [20]:
KNN_preds = []
for i in range(len(predict)):
    _, _, _, rating_pred, _ = gs.predict(predict.userID[i], predict.movieID[i])
    KNN_preds.append(rating_pred)

predict['KNNbase_preds'] = KNN_preds

KNN_df = predict.drop(['rating'], axis=1)
KNN_df.rename(columns={'KNNbase_preds':'rating'}, inplace=True)
KNN_df.to_csv('KNNBase_rating_v1.csv')
KNN_df.round().to_csv('KNNBase_rating_v2.csv')

In [ ]:
#SVD Prediction
SVD_params = {'n_factors': [110, 120, 140, 160], 'n_epochs': [90, 100, 110], 
              'lr_all': [0.001, 0.003, 0.005, 0.008],
              'reg_all': [0.08, 0.1, 0.15]}

gs_svd = GridSearchCV(SVD, SVD_params, measures=['rmse','mae'], cv=3, refit=True)
gs_svd.fit(data)

In [30]:
#SVD Evaluation
SVD_params = {'n_factors': [80, 90, 100, 110, 120, 140], 'n_epochs': [10, 15, 20, 25, 30], 
              'lr_all': [0.001, 0.003, 0.005, 0.008], 'reg_all': [0.01, 0.02, 0.04, 0.8]}

gs_svd = GridSearchCV(SVD, SVD_params, measures=['rmse','mae'], cv=3, refit=True)
gs_svd.fit(data)

In [45]:
print(gs_svd.best_score['rmse'], gs_svd.best_score['mae'])
print(gs_svd.best_params['rmse'], gs_svd.best_params['mae'])

0.9328904725718825 0.7396705942369611
{'n_factors': 80, 'n_epochs': 15, 'lr_all': 0.008, 'reg_all': 0.04} {'n_factors': 80, 'n_epochs': 15, 'lr_all': 0.008, 'reg_all': 0.04}


In [31]:
rmse_svd_algo = gs_svd.best_estimator['rmse']
cross_validate(rmse_svd_algo, data, measures=['rmse', 'mae'], cv = 5, verbose =True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9312  0.9302  0.9334  0.9167  0.9271  0.9277  0.0059  
MAE (testset)     0.7375  0.7382  0.7461  0.7224  0.7306  0.7349  0.0080  
Fit time          1.21    1.20    1.19    1.17    1.19    1.19    0.01    
Test time         0.06    0.05    0.05    0.06    0.05    0.05    0.00    


{'test_rmse': array([0.93118173, 0.93022924, 0.93343234, 0.91666788, 0.92706639]),
 'test_mae': array([0.73747813, 0.73815904, 0.74606448, 0.7223568 , 0.73063615]),
 'fit_time': (1.209122657775879,
  1.2042574882507324,
  1.1910121440887451,
  1.1704533100128174,
  1.1859655380249023),
 'test_time': (0.057245492935180664,
  0.05086660385131836,
  0.0486607551574707,
  0.05747628211975098,
  0.04749894142150879)}

In [32]:
mae_svd_algo = gs_svd.best_estimator['mae']
cross_validate(mae_svd_algo, data, measures=['rmse', 'mae'], cv = 5, verbose =True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9358  0.9242  0.9308  0.9347  0.9259  0.9303  0.0046  
MAE (testset)     0.7456  0.7296  0.7377  0.7406  0.7356  0.7378  0.0053  
Fit time          1.22    1.18    1.19    1.22    1.17    1.19    0.02    
Test time         0.05    0.06    0.05    0.05    0.05    0.05    0.00    


{'test_rmse': array([0.93581428, 0.92418888, 0.93082169, 0.93471749, 0.92586959]),
 'test_mae': array([0.74560234, 0.72958991, 0.73769314, 0.74062014, 0.73560995]),
 'fit_time': (1.2151479721069336,
  1.180882453918457,
  1.1868171691894531,
  1.2178964614868164,
  1.1681089401245117),
 'test_time': (0.046874046325683594,
  0.0557098388671875,
  0.04636836051940918,
  0.053220510482788086,
  0.05458545684814453)}

In [35]:
SVD_preds = []
for i in range(len(predict)):
    _, _, _, svd_rating_pred, _ = best_svd_algo.predict(predict.userID[i], predict.movieID[i])
    SVD_preds.append(svd_rating_pred)

predict['SVD_preds'] = SVD_preds
SVD_Prediction = predict.drop(['rating','KNN_preds'], axis=1)
SVD_Prediction.rename(columns={'SVD_preds':'rating'}, inplace =True)
SVD_Prediction.to_csv('svd_pred2.csv')
SVD_Prediction.round().to_csv('svd_pred_2v2.csv')